In [ ]:
import json
from together import Together
from openai import OpenAI

# togetherai_key = <insert togetherai key>
together_client = Together(api_key=togetherai_key)

# openai_key = <insert openai key>
client = OpenAI(api_key=openai_key)

In [3]:
# Loading all JSONs

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/ref_extractions.json', 'r') as f:
    ref_extractions = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/case_summary_pairs_list.json', 'r') as f:
    case_summary_pairs_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_xml_paths.json', 'r') as f:
    matching_uids_xml_paths = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_extrac_list.json', 'r') as f:
    citation_list = json.load(f)

with open ('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/matching_uids_pmid_list.json', 'r') as f:
    matching_uids_pmid_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/o3_mini_response_list.json', 'r') as f:
    o3_mini_response_list = json.load(f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/indices_list.json', 'r') as f:
    citation_valid_indices = json.load(f)


In [4]:
def calc_indices_summaries_list(case_summary_pairs_list):
    """
    Returns two lists:
      - A list of indices corresponding to elements depicting both cases.
      - A list of indices corresponding to elements depicting less than 2 cases.
    """
    valid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) == 2]
    invalid_indices = [idx for idx in range(len(case_summary_pairs_list)) if len(case_summary_pairs_list[idx]) != 2]

    return valid_indices, invalid_indices



In [5]:
summaries_valid_indices, summaries_invalid_indices = calc_indices_summaries_list(case_summary_pairs_list)

In [22]:
len(summaries_invalid_indices)

83

In [10]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/summaries_valid_indices.json', 'w') as f:
#     json.dump(summaries_valid_indices, f)


with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/summaries_valid_indices.json', 'r') as g:
    summaries_valid_indices = json.load(g)

In [6]:
def find_common_indices(summaries_valid_indices, citation_valid_indices):
    """
    Find the common indices between summaries_valid_indices and citation_valid_indices:
      - summaries_valid_indices - represents the indices corresponding to successfully extracted case summaries (0-31,691) 
      - citation_valid_indices - represents the indices corresponding to successfully cited-citing pairs (0-31,691)
    """
    summaries_valid_indices_set = set(summaries_valid_indices)
    citation_valid_indices_set = set(citation_valid_indices)
    common_indices = []

    for i in range(len(matching_uids_pmid_list)):
        if i in summaries_valid_indices_set and i in citation_valid_indices_set:
            common_indices.append(i)
    
    return common_indices


common_indices = find_common_indices(summaries_valid_indices, citation_valid_indices)

In [13]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/common_indices.json', 'w') as f:
#     json.dump(common_indices, f)


with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/common_indices.json', 'r') as g:
    common_indices = json.load(g)

In [7]:
citation_valid_indices_dict = {citation_valid_indices[idx]: idx for idx in range(len(citation_valid_indices))}

In [ ]:
### citation_valid_indices_dict will not be saved as json because it turns the keys into strings (instead of ints) ###

# # with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_valid_indices_dict.json', 'w') as f:
# #     json.dump(citation_valid_indices_dict, f)


# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/citation_valid_indices_dict.json', 'r') as g:
#     citation_valid_indices_dict = json.load(g)

In [16]:
len(citation_valid_indices_dict)

31080

In [9]:
print(f"The length of case summary pairs list is: {len(case_summary_pairs_list)}")
print(f"The length of o3 response list is: {len(o3_mini_response_list)}")

The length of case summary pairs list is: 31692
The length of o3 response list is: 31080


In [17]:
curr_index = common_indices[6663]

case1 = case_summary_pairs_list[curr_index][0]

case2 = case_summary_pairs_list[curr_index][1]

citation_reason = o3_mini_response_list[citation_valid_indices_dict[curr_index]]

question_generating_prompt = """
You are an experienced clinician specializing in medical imaging. You will be given two cases and the reason for citation between the two (one case report cites the other), and asked to generate a set of comparative, clinically relevant questions that highlight both similarities and differences, helping differentiate potential diagnoses. Your questions should:

1. **Focus on Diagnostic Clues and Clinical Significance:**
   - Avoid trivial or purely technical details (like minor differences in image orientation).
   - Emphasize findings that inform diagnosis (e.g., presence of lesions, pattern of abnormalities, symptoms).

2. **Use a Comparative Framing:**
   - Explicitly compare Case A and Case B.
   - Aim for questions that probe both common ground and distinguishing features.
   - If possible, try asking mostly yes/no questions.

3. **Follow a Hierarchical Reasoning Approach:**
   - **Level 1 (Broad Context)**: Are both cases the same modality/organ system?
   - **Level 2 (General Diagnosis Category)**: Are they both infectious, both neoplastic, etc.?
   - **Level 3 (Specific Features & Findings)**: Detailed imaging findings (e.g., cavitation, consolidation, nodules), clinical presentation, organism type, etc.

4. **Consider Step-by-Step Reasoning:**
   - You may first silently analyze the diagnoses and imaging findings for each case.
   - Then generate questions that a clinician would naturally ask to tease out whether the two cases have the same underlying pathology or different pathologies.

5. **Provide a subtitle specifying the level (1/2/3) of the current set of questions.**
"""

question_generating_user = f"""
Below are two cases and the reason for citation between the two. Read them carefully, then generate a concise list of question prompts (in bullet or numbered format) that compare these cases on clinically meaningful aspects.

---
Case A:
{case1}

Case B:
{case2}

Reason for Citation, Possible Similarities and Explanation:
{citation_reason}
---

Now, **generate the list of comparative, clinically relevant questions** that a radiologist or physician might ask to determine whether these two cases share a diagnosis or differ in key features. 

Remember:
- Maintain clinical depth: mention imaging findings, pathophysiology, or hallmark symptoms.
- Emphasize similarities/differences that impact diagnosis.
- Keep the questions focused and direct, as if you are performing a differential diagnosis.
- Avoid vague or generic questions; ensure each question contributes to understanding diagnostic overlap or divergence.
- Start with broad-level questions (anatomy, broad diagnosis similarity), then proceed to more specific questions about the nature of the condition, causative agents, imaging signs, etc.
- Adhere to yes/no questions, if possible.
- Provide a subtitle specifying the level (1/2/3) of each set of questions.

Provide your final list of questions now.
"""

KeyError: 6822

In [27]:
citation_list[0]

{'citing_title': 'Secondary choroidal neovascularization due to choroidal osteoma after 9\u2009years follow-up',
 'citing_abstract': 'Background Choroidal osteoma is a benign intraocular tumor that can increase risk of developing choroidal neovascularization. The visual prognosis is influenced by the tumor location, decalcification status, overlying RPE atrophy, presence of choroidal neovascularization, persistence of subretinal fluid and occurrence of subretinal hemorrhages. Case presentation The authors present a 40-year-old woman diagnosed with choroidal osteoma of the right eye. Her best corrected visual acuity was 12/20 but decreased to 5/20 due to secondary choroidal neovascularization after 8\u2009years follow up. Fundus examination revealed an enlarged choroidal osteoma in most margins at posterior pole with schistose hemorrhage beside macula. Optical coherence tomography angiography revealed unique features in the vascular changes of choroidal neovascularization in choroidal o

In [11]:
question_generating_prompt = """
You are an experienced clinician specializing in medical imaging. You will be given two cases and the reason for citation between the two (one case report cites the other), and asked to generate a set of comparative, clinically relevant questions that highlight both similarities and differences, helping differentiate potential diagnoses. Your questions should:

1. **Focus on Diagnostic Clues and Clinical Significance:**
   - Avoid trivial or purely technical details (like minor differences in image orientation).
   - Emphasize findings that inform diagnosis (e.g., presence of lesions, pattern of abnormalities, symptoms).

2. **Use a Comparative Framing:**
   - Explicitly compare Case A and Case B.
   - Aim for questions that probe both common ground and distinguishing features.
   - If possible, try asking mostly yes/no questions.

3. **Follow a Hierarchical Reasoning Approach:**
   - **Level 1 (Broad Context)**: Are both cases the same modality/organ system?
   - **Level 2 (General Diagnosis Category)**: Are they both infectious, both neoplastic, etc.?
   - **Level 3 (Specific Features & Findings)**: Detailed imaging findings (e.g., cavitation, consolidation, nodules), clinical presentation, organism type, etc.

4. **Consider Step-by-Step Reasoning:**
   - You may first silently analyze the diagnoses and imaging findings for each case.
   - Then generate questions that a clinician would naturally ask to tease out whether the two cases have the same underlying pathology or different pathologies.

5. **Provide a subtitle specifying the level (1/2/3) of the current set of questions.**
"""

question_generating_user = f"""
Below are two cases and the reason for citation between the two. Read them carefully, then generate a concise list of question prompts (in numbered format) that compare these cases on clinically meaningful aspects.

---
Case A:
%s

Case B:
%s

Reason for Citation, Possible Similarities and Explanation:
%s
---

Now, **generate the list of comparative, clinically relevant questions** that a radiologist or physician might ask to determine whether these two cases share a diagnosis or differ in key features. 

Remember:
- Maintain clinical depth: mention imaging findings, pathophysiology, or hallmark symptoms.
- Emphasize similarities/differences that impact diagnosis.
- Keep the questions focused and direct, as if you are performing a differential diagnosis.
- Avoid vague or generic questions; ensure each question contributes to understanding diagnostic overlap or divergence.
- Start with broad-level questions (anatomy, broad diagnosis similarity), then proceed to more specific questions about the nature of the condition, causative agents, imaging signs, etc.
- Adhere to yes/no questions, if possible.
- Provide a subtitle specifying the level (1/2/3) of each set of questions.

Provide your final list of questions now.
"""

In [12]:
### Check the size of the complete jsonl (before splitting it) ###
import os
file_size = os.path.getsize("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests.jsonl")
print(f"The file is {file_size} bytes.")
print(f"The batch input file should be maximum 209715200 for the o3-mini model, hence we need {int(file_size / 209715200) + 1} batches.")


The file is 338579405 bytes.
The batch input file should be maximum 209715200 for the o3-mini model, hence we need 2 batches.


In [ ]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_A.jsonl", "w", encoding="utf-8") as f:
    for i, curr_index in enumerate(common_indices[:16000], start=1):

        case1 = case_summary_pairs_list[curr_index][0]
        case2 = case_summary_pairs_list[curr_index][1]
        citation_reason = o3_mini_response_list[citation_valid_indices_dict[curr_index]]


        request_obj = {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o3-mini",
                "reasoning_effort": "low",
                "messages": [
                    {"role": "developer", "content": question_generating_prompt},
                    {"role": "user", "content": question_generating_user % (case1, case2, citation_reason)}
                ]
            }
        }
        # Write each object on a new line in the JSONL file
        f.write(json.dumps(request_obj, ensure_ascii=False) + "\n")

In [14]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_A.jsonl", 'r') as g:
    questions_requests_list_A = [json.loads(line) for line in g]

In [16]:
questions_requests_list_A[-1]

{'custom_id': 'request-16000',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'o3-mini',
  'reasoning_effort': 'low',
  'messages': [{'role': 'developer',
    'content': '\nYou are an experienced clinician specializing in medical imaging. You will be given two cases and the reason for citation between the two (one case report cites the other), and asked to generate a set of comparative, clinically relevant questions that highlight both similarities and differences, helping differentiate potential diagnoses. Your questions should:\n\n1. **Focus on Diagnostic Clues and Clinical Significance:**\n   - Avoid trivial or purely technical details (like minor differences in image orientation).\n   - Emphasize findings that inform diagnosis (e.g., presence of lesions, pattern of abnormalities, symptoms).\n\n2. **Use a Comparative Framing:**\n   - Explicitly compare Case A and Case B.\n   - Aim for questions that probe both common ground and distinguishing features.\n   - I

In [79]:
questions_batch_input_file_A = client.files.create(
  file=open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_A.jsonl", "rb"),
  purpose="batch"
)

In [80]:
questions_batch_input_file_id_A = questions_batch_input_file_A.id

client.batches.create(
    input_file_id=questions_batch_input_file_id_A,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Generating Questions part A"
    }
)

Batch(id='batch_67e15447b8808190b7d2ee82817379eb', completion_window='24h', created_at=1742820423, endpoint='/v1/chat/completions', input_file_id='file-8pzxWs2ygeLHv5TQS9H9t4', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1742906823, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Generating Questions part A'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [4]:
client.batches.retrieve("batch_67e15447b8808190b7d2ee82817379eb")

Batch(id='batch_67e15447b8808190b7d2ee82817379eb', completion_window='24h', created_at=1742820423, endpoint='/v1/chat/completions', input_file_id='file-8pzxWs2ygeLHv5TQS9H9t4', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1742825429, error_file_id=None, errors=None, expired_at=None, expires_at=1742906823, failed_at=None, finalizing_at=1742823579, in_progress_at=1742820436, metadata={'description': 'Generating Questions part A'}, output_file_id='file-EyYYFCpYtzgww7FgiKLPDs', request_counts=BatchRequestCounts(completed=16000, failed=0, total=16000))

In [9]:
file_response = client.files.content('file-EyYYFCpYtzgww7FgiKLPDs')
file_response_text = file_response.text

json_strings = file_response_text.strip().splitlines()

# Parse each JSON string into a dictionary and store in a list
response_dicts_A = [json.loads(line) for line in json_strings]

generated_questions_A = [response_dicts_A[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(response_dicts_A))]

In [12]:
print(generated_questions_A[15000])

Level 1: Broad Context and Modality  
1. Do both Case A and Case B utilize CT imaging to evaluate for bowel-related pathology?  
2. Are both cases focused on pathology within the colon?

Level 2: General Diagnosis Category (Gallstone Ileus vs. Other Causes)  
3. Do both cases present with findings suggestive of gallstone ileus resulting in bowel obstruction?  
4. Are both cases associated with a history (or imaging evidence) of gallbladder disease that could predispose to gallstone migration?

Level 3: Specific Imaging Findings and Clinical Features  
5. Do both cases demonstrate the presence of an intraluminal calcified structure (gallstone) causing colonic obstruction?  
6. Is pneumobilia or evidence of a cholecystoenteric fistula present in both cases?  
7. Do both cases show signs of adjacent colonic wall thickening or ischemic changes secondary to the obstructing gallstone?  
8. Were the clinical management decisions (surgical intervention vs. conservative management) influenced b

In [6]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions_A.json', 'w') as f:
#     json.dump(generated_questions_A, f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions_A.json', 'r') as f:
    generated_questions_A = json.load(f)

In [7]:
len(generated_questions_A)

16000

In [17]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_B.jsonl", "w", encoding="utf-8") as f:
    for i, curr_index in enumerate(common_indices[16000:], start=16001):

        case1 = case_summary_pairs_list[curr_index][0]
        case2 = case_summary_pairs_list[curr_index][1]
        citation_reason = o3_mini_response_list[citation_valid_indices_dict[curr_index]]


        request_obj = {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o3-mini",
                "reasoning_effort": "low",
                "messages": [
                    {"role": "developer", "content": question_generating_prompt},
                    {"role": "user", "content": question_generating_user % (case1, case2, citation_reason)}
                ]
            }
        }
        # Write each object on a new line in the JSONL file
        f.write(json.dumps(request_obj, ensure_ascii=False) + "\n")

In [15]:
with open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_B.jsonl", 'r') as g:
    questions_requests_list_B = [json.loads(line) for line in g]

In [ ]:
questions_batch_input_file_B = client.files.create(
  file=open("/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/questions_batch_requests_B.jsonl", "rb"),
  purpose="batch"
)

In [26]:
questions_batch_input_file_id_B = questions_batch_input_file_B.id

client.batches.create(
    input_file_id=questions_batch_input_file_id_B,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "Generating Questions part B"
    }
)

Batch(id='batch_67e15e1f712c8190a32529b1476fb66e', completion_window='24h', created_at=1742822943, endpoint='/v1/chat/completions', input_file_id='file-Qnd24YRejqgz2aE3QTWVhT', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1742909343, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Generating Questions part B'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
client.batches.retrieve("batch_67e15e1f712c8190a32529b1476fb66e")

Batch(id='batch_67e15e1f712c8190a32529b1476fb66e', completion_window='24h', created_at=1742822943, endpoint='/v1/chat/completions', input_file_id='file-Qnd24YRejqgz2aE3QTWVhT', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1742826110, error_file_id=None, errors=None, expired_at=None, expires_at=1742909343, failed_at=None, finalizing_at=1742824911, in_progress_at=1742822949, metadata={'description': 'Generating Questions part B'}, output_file_id='file-EJ3Ee16SzqT3kVCzZowiLk', request_counts=BatchRequestCounts(completed=15002, failed=0, total=15002))

In [10]:
file_response = client.files.content('file-EJ3Ee16SzqT3kVCzZowiLk')
file_response_text = file_response.text

json_strings = file_response_text.strip().splitlines()

# Parse each JSON string into a dictionary and store in a list
response_dicts_B = [json.loads(line) for line in json_strings]

generated_questions_B = [response_dicts_B[i]['response']['body']['choices'][0]['message']['content'] for i in range(len(response_dicts_B))]

In [14]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions_B.json', 'w') as f:
#     json.dump(generated_questions_B, f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions_B.json', 'r') as f:
    generated_questions_B = json.load(f)

In [15]:
len(generated_questions_B)

15002

In [ ]:
# Combining the two lists into a single list
generated_questions = generated_questions_A + generated_questions_B

In [35]:
# with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions.json', 'w') as f:
#     json.dump(generated_questions, f)

with open('/cs/labs/tomhope/yuvalbus/pmc/pythonProject/largeListsGuy/generated_questions.json', 'r') as f:
    generated_questions = json.load(f)

31002